In [48]:
import mysql.connector
from datetime import datetime
import os

# Function to create valid email list and invalid email list
def insertEmails(file_path, db, vendorId):
    # Create a cursor object within the function
    cursor = None  # Initialize cursor variable
    try: 
        with open(file_path, 'r') as file:
            cursor = db.cursor()
            for line in file:
                line = line.strip()  # Remove any leading/trailing whitespace
                if "@" in line:
                    strStatus = 'V'
                    selectSql = "SELECT COUNT(*) FROM vki.emails WHERE email_address = '" + line + "'"
                    cursor.execute(selectSql)
                    selResults = cursor.fetchone()
                    if selResults and selResults[0] > 0:  # Access the count result
                        field_value = selResults[0]  # Get the count (integer)
                        if field_value > 0:
                            strStatus = 'RE'
                    sql = "INSERT INTO vki.emails (email_address, email_aq_dt, email_stat, vendor_id) VALUES (%s, %s, %s, %s)"
                    values = (line, datetime.now(), strStatus, vendorId)
                    cursor.execute(sql, values)
                    db.commit()  # Commit after each insert
                else:
                    sql = "INSERT INTO vki.emails (email_address, email_aq_dt, email_stat, vendor_id) VALUES (%s, %s, %s, %s)"
                    values = (line, datetime.now(), 'R', vendorId)
                    cursor.execute(sql, values)
                    db.commit()  # Commit after each insert
    finally:
        # Close the cursor properly if it was created
        if cursor:
            cursor.close()  
        db.commit()  # Final commit
    return

# Step 1: Connect to the MySQL database
db = mysql.connector.connect(
    host="localhost",
    user="vkiuser",
    password="vkiuser",
    database="vki"
)

folder_path = '/Users/pranityadav/Documents/VKI/VendorEmails'
for filename in os.listdir(folder_path):
    if not filename.startswith('.'):
        file_path = os.path.join(folder_path, filename)
        if os.path.isfile(file_path):
            # Split the filename into name and extension
            name, ext = os.path.splitext(filename)
            if name:  # Ensure the name part is not empty
                last_char = name[-1]
                insertEmails(file_path, db, last_char)

db.close() # Ensure the connection is closed